In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import FactorAnalysis
from functools import partial
import scipy as sp
import scipy.optimize
import seaborn as sb
from scipy.spatial.transform import Rotation as R

In [2]:
data = pd.read_sas('nmtwins.sas7bdat')
data = data.groupby('pairnum').mean()
data = data.drop(['zygosity','sex'],axis = 1).dropna() #we only want:grades, parent's education and family income
data

,moed,faed,faminc,english,math,socsci,natsci,vocab
pairnum,,,,,,,,
1.0,3.0,4.0,2.0,12.5,13.5,16.0,14.0,13.0
4.0,1.0,1.0,1.0,18.5,19.5,14.5,14.5,13.5
5.0,1.0,1.0,1.0,13.5,10.5,14.0,12.0,13.5
7.0,3.0,2.0,4.0,8.5,17.5,11.0,13.5,8.5
10.0,5.0,4.0,3.0,15.0,18.0,18.0,20.5,20.0
...,...,...,...,...,...,...,...,...
1500.0,1.0,4.0,2.0,20.0,26.5,22.0,20.0,22.5
1501.0,1.0,1.0,1.0,17.5,21.5,23.5,22.0,22.5
1503.0,3.0,4.0,4.0,17.0,22.5,25.0,22.0,26.0


In [3]:
def loadings_table(loadings, variables):
    return pd.DataFrame(np.round(loadings,2),columns=['F' + str(i) for i in range(loadings.shape[1])],index=variables)

In [4]:
sigma = data.corr()
eigval, eigvec = np.linalg.eig(sigma)
loadings_pca = (eigvec * np.sqrt(eigval))[:, :3]

In [5]:
loadings_table(loadings_pca, data.columns)

,F0,F1,F2
moed,-0.41,-0.69,-0.51
faed,-0.51,-0.70,-0.05
faminc,-0.44,-0.64,0.58
english,-0.80,0.23,0.00
math,-0.81,0.16,0.02
socsci,-0.87,0.26,-0.00
natsci,-0.83,0.29,-0.07
vocab,-0.86,0.16,0.01


In [ ]:
#rotate_3d(L,rotation_vector):
def V(L_star_tilde):
    p = L_star_tilde.shape[0]
    return -sum((L_star_tilde ** 4).sum(axis = 1)-(L_star_tilde ** 2).sum(axis = 1) ** 2 / p) /p
#v_3d